In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('new_dataframe.csv')
df.head(5)


,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
0,0,867689,12,iOS,business,no surge,1,1,1,1,0,0,7.982135,20,low
1,1,752172,5,Android,economy,no surge,1,1,1,1,0,1,2.908468,27,high
2,2,486559,15,Android,comfort,no surge,1,1,1,1,0,0,7.224614,21,high
3,3,304024,0,Android,economy,no surge,1,1,1,1,0,1,1.874349,52,low
4,4,139420,0,Android,business,no surge,1,1,1,1,0,0,10.704778,19,low


# Для устройств Android наблюдаем более высокие юзерские отмены
гипотеза: Если изменить дизайн приложения для Android, то количество отказов от поездок на устройствах Android уменьшится в 2 раза 

In [ ]:
# создаю новый датафрейм, где преставлены только Android юзеры
df_new = df[df['os'].isin(['Android'])]
df_new.head(5)

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
1,1,752172,5,Android,economy,no surge,1,1,1,1,0,1,2.908468,27,high
2,2,486559,15,Android,comfort,no surge,1,1,1,1,0,0,7.224614,21,high
3,3,304024,0,Android,economy,no surge,1,1,1,1,0,1,1.874349,52,low
4,4,139420,0,Android,business,no surge,1,1,1,1,0,0,10.704778,19,low
5,5,139455,5,Android,comfort,NaN,1,0,0,0,0,1,NaN,24,high


In [ ]:
# группирую датафрейм поюзерно, и считаю сумму по заказам и отменам
df_new = df_new.groupby(['user_id']).agg({'order_made': "sum", 'user_cancelled': "sum"})
df_new.head(10)

,order_made,user_cancelled
user_id,,
100093,2,0
100118,2,1
100134,3,1
100166,4,1
100255,1,1
100275,1,0
100303,1,0
100313,2,1
100375,3,2


In [ ]:
# Добавляю новый столбец в датафрейм - конверсию в отказы (user_cancelled / order_made)
df_new['cancel_conv'] = df_new['user_cancelled']/df_new['order_made'] * 100
df_new.head(10)

,order_made,user_cancelled,cancel_conv
user_id,,,
100093,2,0,0.000000
100118,2,1,50.000000
100134,3,1,33.333333
100166,4,1,25.000000
100255,1,1,100.000000
100275,1,0,0.000000
100303,1,0,0.000000
100313,2,1,50.000000
100375,3,2,66.666667


In [ ]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22154 entries, 100093 to 999978
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_made      22154 non-null  int64  
 1   user_cancelled  22154 non-null  int64  
 2   cancel_conv     19527 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 692.3 KB


In [ ]:
# оставляю только тех пользователей, которые делали отмены (убираю нули)
df_new = df_new[df_new['cancel_conv'] > 0]
df_new.head(5)

,order_made,user_cancelled,cancel_conv
user_id,,,
100118,2,1,50.000000
100134,3,1,33.333333
100166,4,1,25.000000
100255,1,1,100.000000
100313,2,1,50.000000


In [ ]:
# в обновленном датасете осталось 6345 строк
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6345 entries, 100118 to 999878
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_made      6345 non-null   int64  
 1   user_cancelled  6345 non-null   int64  
 2   cancel_conv     6345 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 198.3 KB


In [ ]:
import numpy as np

In [ ]:
# добавляю новую колонку, где каждому юзеру присвается рандомное значение от 0 до 1
df_new['rand'] = np.random.random(size=(6345))
df_new.describe()

,order_made,user_cancelled,cancel_conv,rand
count,6345.000000,6345.000000,6345.000000,6345.000000
mean,2.449961,1.158865,58.034973,0.504908
std,1.217022,0.409122,28.109487,0.289406
min,1.000000,1.000000,10.000000,0.000066
25%,2.000000,1.000000,33.333333,0.252550
50%,2.000000,1.000000,50.000000,0.506455
75%,3.000000,1.000000,100.000000,0.757965
max,10.000000,4.000000,100.000000,0.999682


In [ ]:
# формирую 2 выборки на основе столбца rand 
df_1 = df_new[df_new['rand'] < 0.5]
df_2 = df_new[df_new['rand'] > 0.5]

In [ ]:
from scipy.stats import mannwhitneyu

alpha = 0.05  

In [ ]:
# проверяю на однородность с помощью рангового критерия Манна-Уитни
result = mannwhitneyu(df_1['cancel_conv'], df_2['cancel_conv'])
print('p-value: %.8f' % result.pvalue)

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу')
else:
    print('Не отвергаем нулевую гипотезу')

p-value: 0.34728159
Не отвергаем нулевую гипотезу


# В период Surge видим более низкую price-to-order конверсию
Гипотеза: если в период Surge ввести кэшбек за совершенную поездку, то это повысит price-to-order конверсию на 10%

In [ ]:
# создаю новый датафрейм, где преставлен только период Surge
df_new2 = df[df['surge'].isin(['surge'])]
df_new2.head(5)

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
9,9,662775,17,Android,comfort,surge,1,1,1,1,0,1,4.828326,36,low
10,10,464055,20,iOS,comfort,surge,1,1,1,1,0,0,3.647097,25,high
18,18,393096,23,Android,comfort,surge,1,1,0,0,0,1,2.523357,19,high
21,21,103860,13,iOS,economy,surge,1,1,1,1,0,1,7.934002,21,high
24,24,461185,21,iOS,economy,surge,1,1,1,1,0,1,1.590209,29,high


In [ ]:
# группирую датафрейм поюзерно, и считаю сумму по заказам и отменам
df_new2 = df_new2.groupby(['user_id']).agg({'order_made': "sum", 'price_seen': "sum"})
df_new2.head(10)

,order_made,price_seen
user_id,,
100093,1,2
100118,1,1
100134,1,1
100166,2,2
100255,0,2
100275,0,1
100313,3,3
100375,4,4
100405,2,4


In [ ]:
# Добавляю новый столбец в датафрейм - конверсию price-to-order (order_made / price_seen)
df_new2['price_order'] = df_new2['order_made']/df_new2['price_seen'] * 100
df_new2.describe()

,order_made,price_seen,price_order
count,17780.000000,17780.000000,17780.000000
mean,1.286445,1.779696,72.352101
std,0.919082,0.962777,37.807172
min,0.000000,1.000000,0.000000
25%,1.000000,1.000000,50.000000
50%,1.000000,2.000000,100.000000
75%,2.000000,2.000000,100.000000
max,7.000000,8.000000,100.000000


In [ ]:
# оставляю только тех пользователей, которые делали отмены (убираю нули)
df_new2 = df_new2[df_new2['price_order'] > 0]
df_new2.head(5)

,order_made,price_seen,price_order
user_id,,,
100093,1,2,50.0
100118,1,1,100.0
100134,1,1,100.0
100166,2,2,100.0
100313,3,3,100.0


In [ ]:
# в обновленном датасете осталось 14845 строк
df_new2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14845 entries, 100093 to 999945
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   order_made   14845 non-null  int64  
 1   price_seen   14845 non-null  int64  
 2   price_order  14845 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 463.9 KB


In [ ]:
# добавляю новую колонку, где каждому юзеру присвается рандомное значение от 0 до 1
df_new2['rand'] = np.random.random(size=(14845))
df_new2.describe()

,order_made,price_seen,price_order,rand
count,14845.000000,14845.000000,14845.000000,14845.000000
mean,1.540788,1.893904,86.656811,0.496918
std,0.787284,0.993440,21.732923,0.288865
min,1.000000,1.000000,20.000000,0.000014
25%,1.000000,1.000000,66.666667,0.245179
50%,1.000000,2.000000,100.000000,0.493386
75%,2.000000,2.000000,100.000000,0.746460
max,7.000000,8.000000,100.000000,0.999906


In [ ]:
# формирую 2 выборки на основе столбца rand 
df_3 = df_new2[df_new2['rand'] < 0.5]
df_4 = df_new2[df_new2['rand'] > 0.5]

In [ ]:
# проверяю на однородность с помощью рангового критерия Манна-Уитни
result = mannwhitneyu(df_3['price_order'], df_4['price_order'])
print('p-value: %.8f' % result.pvalue)

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу')
else:
    print('Не отвергаем нулевую гипотезу')

p-value: 0.71578691
Не отвергаем нулевую гипотезу
